#   Crawling PTA Trunojoyo Prodi Manajemen

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, sys, time
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
BASE_URL = "https://pta.trunojoyo.ac.id/c_search/byprod"

In [3]:
def print_progress(prodi_id, prodi, current_page, total_pages):
    percent = (current_page / total_pages) * 100
    bar_length = 20
    filled_length = int(bar_length * current_page // total_pages)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f'\r[{prodi_id}] {prodi} - Page {current_page}/{total_pages} [{bar}] {percent:.2f}%')
    sys.stdout.flush()
    if current_page == total_pages:
        sys.stdout.write('\n')

def crawl_detail(link_keluar, prodi):
    try:
        id_match = re.search(r"/detail/(\d+)", link_keluar)
        pta_id = id_match.group(1) if id_match else None

        response = requests.get(link_keluar, timeout=10)
        soup1 = BeautifulSoup(response.content, "html.parser")
        isi = soup1.select_one('div#content_journal')

        judul = isi.select_one('a.title').text.strip()
        penulis = isi.select_one('span:contains("Penulis")').text.split(' : ')[1]
        pembimbing_pertama = isi.select_one('span:contains("Dosen Pembimbing I")').text.split(' : ')[1]
        pembimbing_kedua = isi.select_one('span:contains("Dosen Pembimbing II")').text.split(' :')[1]

        paragraf = isi.select('p[align="justify"]')
        abstrak_id = paragraf[0].get_text(strip=True) if len(paragraf) > 0 else "N/A"
        abstrak_en = paragraf[1].get_text(strip=True) if len(paragraf) > 1 else "N/A"

        return {
            "id": pta_id,
            "penulis": penulis,
            "judul": judul,
            "abstrak_id": abstrak_id,
            "abstrak_en": abstrak_en,
            "pembimbing_pertama": pembimbing_pertama,
            "pembimbing_kedua": pembimbing_kedua,
            "prodi": prodi,
        }
    except Exception as e:
        return None


##  Fungsi untuk menentukan maksimal halaman pada daftar page pada website

In [4]:
def get_max_page(prodi_id):
    url = f"{BASE_URL}/{prodi_id}/1"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")

    last_page = soup.select_one('ol.pagination a:contains("»")')
    if last_page and "href" in last_page.attrs:
        href = last_page["href"]
        max_page = int(href.split("/")[-1])
        return max_page
    return 1

## Fungsi Crawling Khusus Program Studi Manajemen

In [5]:
def pta_manajemen_only():
    start_time = time.time()
    results = []

    # Hanya crawling program studi Manajemen (byprod 7)
    prodi_id = 7
    max_page = get_max_page(prodi_id)

    print(f"🚀 Memulai crawling data Program Studi Manajemen...")
    print(f"📄 Total halaman yang akan di-crawl: {max_page}")
    print(f"🔗 URL: {BASE_URL}/{prodi_id}")
    print("-" * 50)

    for j in range(1, max_page + 1):
        url = f"{BASE_URL}/{prodi_id}/{j}"
        r = requests.get(url)
        soup = BeautifulSoup(r.content, "html.parser")
        jurnals = soup.select('li[data-cat="#luxury"]')

        isii = soup.select_one('div#begin')
        if not isii:
            continue
        prodi_full = isii.select_one('h2').text.strip()
        prodi = prodi_full.replace("Journal Jurusan ", "")

        links = [jurnal.select_one('a.gray.button')['href'] for jurnal in jurnals]

        # ambil detail paralel
        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_link = {executor.submit(crawl_detail, link, prodi): link for link in links}
            for future in as_completed(future_to_link):
                data = future.result()
                if data:
                    results.append(data)

        # progress bar
        print_progress(prodi_id, prodi, j, max_page)

    sys.stdout.write("\n")

    df = pd.DataFrame(results)
    df.to_csv("pta_manajemen.csv", index=False, encoding="utf-8-sig")

    end_time = time.time()
    elapsed = int(end_time - start_time)
    jam, sisa = divmod(elapsed, 3600)
    menit, detik = divmod(sisa, 60)

    print("\n✅ Data Program Studi Manajemen berhasil dikumpulkan!")
    print(f"📊 Total entri: {len(df)}")
    print(f"⏱️ Waktu eksekusi: {jam} jam {menit} menit {detik} detik")
    print(f"💾 Data tersimpan di: pta_manajemen.csv")

    return df


In [6]:
# Jalankan crawling hanya untuk Program Studi Manajemen
pta_manajemen_only()


c:\Users\LX\AppData\Local\Programs\Python\Python313\Lib\site-packages\soupsieve\css_parser.py:862: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


🚀 Memulai crawling data Program Studi Manajemen...
📄 Total halaman yang akan di-crawl: 207
🔗 URL: https://pta.trunojoyo.ac.id/c_search/byprod/7
--------------------------------------------------
[7] Manajemen - Page 207/207 [████████████████████] 100.00%


✅ Data Program Studi Manajemen berhasil dikumpulkan!
📊 Total entri: 1021
⏱️ Waktu eksekusi: 0 jam 17 menit 49 detik
💾 Data tersimpan di: pta_manajemen.csv


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,080211100050,Wahyu Kurniawan,PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...,,,"Dr. Dra. Hj. Iriani Ismail, MM","Dra. Hj. S. Anugrahini Irawati, MM",Manajemen
1,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
2,080211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,Abstract\r\nThis research use quantitative met...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
3,090211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,This study wanted to know the brand associatio...,Nurita Andriani,Yustina Chrismardani,Manajemen
4,080211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...",ABSTRACT\r\n\r\nIn an effort to increase labor...,"Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
...,...,...,...,...,...,...,...,...
1016,160211100064,ACH FATHONI,PERAN SERVICE PERFORMANCE DAN CLIMATE ORGANIZA...,ABSTRAK\nTujuan dari penelitian ini adalah unt...,ABSTRACK\n The purpose of this study...,"YUDHI PRASETYA MADA, S.E., M.M.",,Manajemen
1017,160211100071,Husnul Hotimah,Analisis Cost Volume Profit Untuk Menentukan T...,ABSTRAK\nPenelitian ini bertujuan untuk menget...,ABSTRACT\nThis study aims to determine the cal...,"Hj. Evaliati Amaniyah, S.E., M.S.M.",,Manajemen
1018,160211100030,INTAN YULLIA NINGSIH,BAURAN PROMOSI PADA DEALER YAMAHA TRETAN MOTOR...,ABSTRAK\nPenelitian ini bertujuan: (1) Untuk m...,ABSTRACK\nThis study aims: (1) To find out whe...,"DR. MOHAMMAD ARIEF, S.E., M.M.",,Manajemen
1019,160211100041,Novia Prihastiwi Ningtyas,"ANALISA PENGARUH RASIO BOPO, LDR, ROA TERHADAP...",Tujuan penelitian ini adalah untuk mengetahui ...,The purpose of this study was to investigate t...,"Hj. Evaliati Amaniyah, S.E., M.S.M",,Manajemen


In [7]:
# Load dan tampilkan data hasil crawling Manajemen
import pandas as pd
df_manajemen = pd.read_csv("pta_manajemen.csv")
print(f"📊 Total data Program Studi Manajemen: {len(df_manajemen)}")
print(f"📋 Kolom yang tersedia: {list(df_manajemen.columns)}")
print("\n🔍 Preview data:")
df_manajemen.head()


📊 Total data Program Studi Manajemen: 1021
📋 Kolom yang tersedia: ['id', 'penulis', 'judul', 'abstrak_id', 'abstrak_en', 'pembimbing_pertama', 'pembimbing_kedua', 'prodi']

🔍 Preview data:


,id,penulis,judul,abstrak_id,abstrak_en,pembimbing_pertama,pembimbing_kedua,prodi
0,80211100050,Wahyu Kurniawan,PENGARUH GAYA KEPEMIMPINAN DEMOKRATIK TERHADAP...,NaN,NaN,"Dr. Dra. Hj. Iriani Ismail, MM","Dra. Hj. S. Anugrahini Irawati, MM",Manajemen
1,100211200002,Muhammad Zakaria Utomo,Pengukuran Website Quality Pada Situs Sistem A...,Aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Manajemen
2,80211100044,Hendri Wahyudi Prayitno,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Abstrak\r\nPenelitian ini menggunakan metode k...,Abstract\r\nThis research use quantitative met...,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Manajemen
3,90211200001,Faishal,ANALISIS PERSEPSI BRAND ASSOCIATION MENURUT PE...,Tujuan penelitian ini adalah untuk mengetahui ...,This study wanted to know the brand associatio...,Nurita Andriani,Yustina Chrismardani,Manajemen
4,80211100070,SATIYAH,PENGARUH FAKTOR-FAKTOR PELATIHAN DAN PENGEMBAN...,"ABSTRAK\r\nSatiyah, Pengaruh Faktor-faktor Pel...",ABSTRACT\r\n\r\nIn an effort to increase labor...,"Dra. Hj. S. Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST.SE,M.MT",Manajemen
